In [ ]:
import psycopg2
import pandas as pd

In [ ]:
dbhost = "localhost"
dbname = "db_01"
dbuser = "postgres"
dbpassword = "ABcd@12#$"

# 1/4 train, dev and test difference
# source = 'creditcard_train'
source = 'creditcard_dev'
# source = 'creditcard_test'

In [ ]:
file_data = pd.read_csv(f"{source}.csv")
print(len(file_data))

try:
    conn = psycopg2.connect(dbname=dbname, user=dbuser, password=dbpassword, host=dbhost)
except psycopg2.Error as e:
    print("Error connecting to database:", e)
    exit()

if(conn):
    print('connected')

cur = conn.cursor()

In [ ]:
Arr=[]
for i in range (1, 29):
    Arr.append(f"V{i}")

columns = ", ".join(Arr)
creation_columns=" DECIMAL, ".join(Arr)

In [ ]:
# cur.execute(f"DROP TABLE {source};")

# cur.execute(f"SELECT * FROM {source};")
# for row in cur.fetchall():
#     print(row, end='\n')

# 2/4 train, dev and test difference
cur.execute(f"CREATE TABLE {source} (Time INTEGER, {creation_columns} DECIMAL, Amount INTEGER, Class INTEGER);") #train and dev
# cur.execute(f"CREATE TABLE {source} (Time INTEGER, {creation_columns} DECIMAL, Amount INTEGER);") #test

# 3/4 train, dev and test difference
insert_query = f"INSERT INTO {source} (Time, {columns}, Amount, Class) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);" #train and dev
# insert_query = f"INSERT INTO {source} (Time, {columns}, Amount) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);" #test

In [ ]:
def toSql(dataToSend):
    try:
        cur.execute(insert_query, dataToSend)
        conn.commit()
    except psycopg2.Error as e:
        print("Error inserting data:", e)

In [ ]:
data=[]

for i in range(0, len(file_data)):
    temp=file_data.iloc[i]
    # print(temp)
    t=[]
    # 4/4 train, dev and test difference
    for j in range (0, 31): #train and dev
    # for j in range (0, 30): #test
        t.append(temp[j])
    # print(t)
    toSql(t)

#creditcard_train 6m 11.5s :)
#creditcard_dev m 55.6s ...
#creditcard_test 55.9s

In [ ]:
cur.execute(f"SELECT * FROM {source};")
for row in cur.fetchall():
    print(row, end='\n')

In [ ]:
conn.commit()
cur.close()
conn.close()